In [1]:
from simpletransformers.classification import ClassificationModel
import pandas as pd
from sklearn.model_selection import train_test_split

In [9]:
data = pd.read_csv('finnews_combine_tra.csv', encoding = 'utf-8-sig', header=0)

In [ ]:
# # 分割訓練集、驗證集和測試集
# train_data, eval_data = train_test_split(data, test_size=0.1, random_state=42)
# eval_data, test_data = train_test_split(eval_data, test_size=0.5, random_state=42)

In [10]:
# 分割訓練集和驗證集
train_data, eval_data = train_test_split(data, test_size=0.1, random_state=42)

# 轉換標籤為整數形式（如果需要）
train_data['label'] = train_data['label'].astype(int)
eval_data['label'] = eval_data['label'].astype(int)

# 版本二
train_data = train_data.rename(columns={'label': 'text', 'review': 'label'})
eval_data = eval_data.rename(columns={'label': 'text', 'review': 'label'})

In [11]:
# 創建 ClassificationModel
model = ClassificationModel(
    'bert',
    'bert-base-chinese',
    num_labels=2,  # 情緒分析分為兩個類別，根據需要進行調整
    args={'overwrite_output_dir': True},
    use_cuda=False # True
)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-chinese and are newly i

In [ ]:
# # 訓練模型
# model.train_model(train_data, eval_df=eval_data)

In [12]:
# 訓練模型
model.train_model(train_data, eval_df=eval_data, label='label', text='review')

c:\Users\user\anaconda3\lib\site-packages\simpletransformers\classification\classification_model.py:612: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/159262 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/19908 [00:00<?, ?it/s]

(19908, 0.14455823224886272)

In [19]:
model.model.save_pretrained("D:\\User_Data\\Desktop\\finbert_zh")
model.tokenizer.save_pretrained("D:\\User_Data\\Desktop\\finbert_zh")
model.config.save_pretrained("D:\\User_Data\\Desktop\\finbert_zh")

In [2]:
model_zh = ClassificationModel("bert","D:\\User_Data\\Desktop\\finbert_zh", use_cuda=False)


In [18]:
result, model_output, wrong_predictions = model.eval_model(eval_data)
result

c:\Users\user\anaconda3\lib\site-packages\simpletransformers\classification\classification_model.py:1454: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/17696 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/2212 [00:00<?, ?it/s]

{'mcc': 0.9684243526496912,
 'tp': 9528,
 'tn': 7891,
 'fp': 145,
 'fn': 132,
 'auroc': 0.996747001330452,
 'auprc': 0.9971869731099849,
 'eval_loss': 0.07657575048111542}

In [27]:
# 在測試資料上進行預測
eval_texts = eval_data['review'].tolist()
predictions, raw_outputs = model.predict(eval_texts)

# 顯示預測結果
for text, prediction in zip(eval_texts, predictions):
    print(f'Text: {text}')
    print(f'Predicted Label: {prediction}')
    print('--------------------------')

  0%|          | 0/17696 [00:00<?, ?it/s]

  0%|          | 0/2212 [00:00<?, ?it/s]

Text: 威創股份：與百度戰畧合作 共同打造智幼慧教行業解決方案
Predicted Label: 1
--------------------------
Text: 證通電子擬非公開發行不超億元公司債償債補血等
Predicted Label: 1
--------------------------
Text: High Lift Holding:預計2018年凈損失將達到15億至20億尤安
Predicted Label: 0
--------------------------
Text: 四川產業振興發展投資基金擬堅持 天士力跌停
Predicted Label: 0
--------------------------
Text: 縱橫信通持股股東東證昭悳擬減持不超股份
Predicted Label: 0
--------------------------
Text: Guizhou Bring計劃投資數十億美元建造三個研討會咊支持工程項目
Predicted Label: 1
--------------------------
Text: 今世股東上海銘大擬減持不超2%股份
Predicted Label: 0
--------------------------
Text: 中天能源：擬收購海外油氣資源搨展產業鏈5月19日復牌
Predicted Label: 1
--------------------------
Text: 華沙在人民生活銀行擁有1300萬份股票
Predicted Label: 0
--------------------------
Text: 探路公司股票可能被暫停上市
Predicted Label: 0
--------------------------
Text: 訢控股：京南公司ä溢價20受讓5.8%公司股份
Predicted Label: 1
--------------------------
Text: 豐東股份：持古11.81%股東深圳市興創源投資有限公司擬6個月內堅持不超過1.4
Predicted Label: 0
--------------------------
Text: 揚颿材新：一報季預增151%-180%
Predicted Label: 1
----------------

In [4]:
# 讀取CSV檔案
data = pd.read_csv('news_tw.csv', encoding = 'utf-8-sig')

# 提取需要分析的文本資料
data_to_analyze = data['title'].tolist()

# 使用模型進行預測
predictions, raw_outputs = model_zh.predict(data_to_analyze)

# 評估整體趨勢的算法
def evaluate_trend(predictions):
    total_count = len(predictions)
    positive_count = sum(predictions)
    negative_count = total_count - positive_count

    positive_ratio = positive_count / total_count
    negative_ratio = negative_count / total_count

    if positive_ratio >= 0.7:
        trend = "正面"
    elif positive_ratio <= 0.3:
        trend = "負面"
    else:
        trend = "中性"

    return positive_count, positive_ratio, negative_count, negative_ratio, trend

# 顯示預測結果
for text, prediction in zip(data_to_analyze, predictions):
    print(f'Text: {text}')
    print(f'Predicted Label: {prediction}')
    print('--------------------------')

  0%|          | 0/203 [00:00<?, ?it/s]

  0%|          | 0/26 [00:00<?, ?it/s]

Text: 外資15日買超前十大 人氣王紅不讓 2天13萬張
Predicted Label: 1
--------------------------
Text: 投信15日買超前十大 電金傳通吃、榜首唯一買上萬張
Predicted Label: 1
--------------------------
Text: 三大法人15日賣超前十大 最痛是它、連8日賣逾21萬張
Predicted Label: 1
--------------------------
Text: 三大法人15日買超前十大 一哥2天都封王、狂抱15萬張
Predicted Label: 1
--------------------------
Text: 投信15日賣超前十大 小老虎連砍5刀 它終結6連買照飆
Predicted Label: 0
--------------------------
Text: 日盛投信：選股不選市 台股中長期表現不看淡
Predicted Label: 1
--------------------------
Text: 外資15日賣超前十大 痛宰新光金 累計大砍21萬張
Predicted Label: 0
--------------------------
Text: 那指與標普500連6紅；雙創13個月新高
Predicted Label: 1
--------------------------
Text: AI發威美股續漲，觀察週五盤中台股接手力道
Predicted Label: 1
--------------------------
Text: 投信15日賣超股：群創、台玻、聯電
Predicted Label: 1
--------------------------
Text: 外資15日賣超股：新光金、緯創、開發金
Predicted Label: 1
--------------------------
Text: 三大法人15日買超股：華航、聯電、彰銀
Predicted Label: 1
--------------------------
Text: 外資15日買超股：華航、聯電、大成鋼
Predicted Label: 1
--------------------------
Text: 投信15日買超股：長榮航、緯創

In [5]:
# 評估整體趨勢
positive_count, positive_ratio, negative_count, negative_ratio, trend = evaluate_trend(predictions)
print(f"正面筆數：{positive_count}，佔比：{positive_ratio * 100:.2f}%")
print(f"負面筆數：{negative_count}，佔比：{negative_ratio * 100:.2f}%")
print(f"整體趨勢：{trend}")

正面筆數：128，佔比：63.05%
負面筆數：75，佔比：36.95%
整體趨勢：中性
